In [ ]:
import numpy as np
import pickle as pkl
from database_tools.processing.cardiac import estimate_pulse_rate, estimate_spo2
from database_tools.processing.detect import detect_peaks
from database_tools.tools.dataset import ConfigMapper
from plotly.subplots import make_subplots

from gcp_utils.tools.preprocess import _clean_frame, _split_frame
from gcp_utils.tools.predict import predict_cardiac_metrics

In [ ]:
cm = ConfigMapper('/home/cam/Documents/gcp_utils/gcp_utils/config.ini')

with open('bpm-frame.pkl', 'rb') as f:
    data = pkl.load(f)

red_frame = data['red_frame']
ir_frame = data['ir_frame']

red_frame = np.array(red_frame, dtype=np.float32)
red_frame[np.isnan(red_frame)] = 0
ir_frame = np.array(ir_frame, dtype=np.float32)
ir_frame[np.isnan(ir_frame)] = 0

# 2
red_clean = _clean_frame(red_frame, thresh=cm.deploy.clean_thresh)
ir_clean = _clean_frame(ir_frame, thresh=cm.deploy.clean_thresh)

In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_scatter(y=red_clean, row=1, col=1, name='red')
fig.add_scatter(y=ir_clean, row=2, col=1, name='ir')

In [ ]:
predict_cardiac_metrics(red_clean, ir_clean, cm=cm)

In [ ]:
red_idx, ir_idx = predict_cardiac_metrics(red_clean, ir_clean, cm=cm)
red_peaks, red_troughs = red_idx.values()
ir_peaks, ir_troughs = ir_idx.values()

In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_scatter(y=red_clean, row=1, col=1, name='red')
fig.add_scatter(x=red_peaks, y=red_clean[red_peaks], row=1, col=1, mode='markers', name='red')
fig.add_scatter(x=red_troughs, y=red_clean[red_troughs], row=1, col=1, mode='markers', name='red')

fig.add_scatter(y=ir_clean, row=2, col=1, name='red')
fig.add_scatter(x=ir_peaks, y=ir_clean[ir_peaks], row=2, col=1, mode='markers', name='ir')
fig.add_scatter(x=ir_troughs, y=ir_clean[ir_troughs], row=2, col=1, mode='markers', name='ir')